# LangGraph Single Tool Execution

Understanding how a complete tool execution cycle works in LangGraph.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand the complete message flow** for single tool execution from user query to final response
2. **Examine different message types** (HumanMessage, AIMessage with tool_calls, ToolMessage, AIMessage with content)
3. **Trace the execution cycle** through the graph nodes (LLM → Tools → LLM → End)

## 1. Environment Setup

In [1]:
# Environment setup
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [2]:
# Mermaid helper for graph visualization
def render_mermaid(diagram_code, width=400):
    """Render Mermaid diagrams using mermaid.ink service."""
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

print("✅ Visualization helper defined")

✅ Visualization helper defined


## 2. Define Tools

In [3]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


## 3. Initialize LLM with Tools

In [4]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

✅ LLM initialized with tools


## 4. Build Graph and Compile Application

In [5]:
# Build graph
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


In [6]:
# Visualize graph
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram, width=200)

## 5. Test Case 1: Currency Conversion

Execute a simple currency conversion and trace the message flow.

In [7]:
# Test Case 1: Currency Conversion
state = {
    "messages": [HumanMessage(content="What is 1000 USD in EUR?")]
}

print("Query: What is 1000 USD in EUR?")
print("=" * 70)

result = app.invoke(state)
print(f"\nTotal messages: {len(result['messages'])}")
print("=" * 70)

Query: What is 1000 USD in EUR?

Total messages: 4


In [8]:
result["messages"][-1].content

'1000 USD is 920 EUR.'

### Message Flow Analysis

Let's examine each message in the conversation to understand the complete execution cycle.

In [9]:
# Visualize graph
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram, width=200)

In [10]:
# Examine message flow
print("MESSAGE FLOW ANALYSIS")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 70}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 70}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"  🤖 LLM DECISION: Call tool(s)")
            for tc in msg.tool_calls:
                print(f"    • Tool: {tc['name']}")
                print(f"      Args: {tc['args']}")
        else:
            print(f"  🤖 LLM RESPONSE: Final answer")
            print(f"  Content: {msg.content}")
            
    elif isinstance(msg, ToolMessage):
        print(f"  🔧 TOOL RESULT")
        print(f"  Content:\n{msg.content}")

MESSAGE FLOW ANALYSIS

──────────────────────────────────────────────────────────────────────
MESSAGE 1: HumanMessage
──────────────────────────────────────────────────────────────────────
  👤 USER INPUT
  Content: What is 1000 USD in EUR?

──────────────────────────────────────────────────────────────────────
MESSAGE 2: AIMessage
──────────────────────────────────────────────────────────────────────
  🤖 LLM DECISION: Call tool(s)
    • Tool: currency_converter
      Args: {'amount': 1000, 'to_currency': 'EUR', 'from_currency': 'USD'}

──────────────────────────────────────────────────────────────────────
MESSAGE 3: ToolMessage
──────────────────────────────────────────────────────────────────────
  🔧 TOOL RESULT
  Content:
Conversion Result:
  1,000.00 USD = 920.00 EUR
  Exchange Rate: 1 USD = 0.9200 EUR

──────────────────────────────────────────────────────────────────────
MESSAGE 4: AIMessage
──────────────────────────────────────────────────────────────────────
  🤖 LLM RESPONSE: F

### Deep Dive: Individual Messages

Examine the structure of each message type in detail.

In [11]:
# Deep dive into each message
print("MESSAGE 1: HumanMessage")
print("=" * 70)
print(f"Content: {result['messages'][0].content}")

MESSAGE 1: HumanMessage
Content: What is 1000 USD in EUR?


In [12]:
type(result['messages'][0])

langchain_core.messages.human.HumanMessage

In [13]:
dict(result['messages'][0])

{'content': 'What is 1000 USD in EUR?',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': '51ef0bc3-7c4e-4872-8f5a-dbe633d8eee4'}

In [14]:
# Message 2: AIMessage with tool_calls
print("MESSAGE 2: AIMessage (Tool Call Request)")
print("=" * 70)
msg2 = result['messages'][1]
print(f"Has tool_calls: {bool(msg2.tool_calls)}")
print(f"\nTool Calls Detail:")
print(msg2.tool_calls)

MESSAGE 2: AIMessage (Tool Call Request)
Has tool_calls: True

Tool Calls Detail:
[{'name': 'currency_converter', 'args': {'amount': 1000, 'to_currency': 'EUR', 'from_currency': 'USD'}, 'id': '2a1679f3-365a-47fd-8533-64724a1461cd', 'type': 'tool_call'}]


In [15]:
type(msg2)

langchain_core.messages.ai.AIMessage

In [16]:
dict(msg2)

{'content': '',
 'additional_kwargs': {'function_call': {'name': 'currency_converter',
   'arguments': '{"amount": 1000, "to_currency": "EUR", "from_currency": "USD"}'}},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c32bc-3e3f-7352-94bc-fb61919332ab-0',
 'tool_calls': [{'name': 'currency_converter',
   'args': {'amount': 1000, 'to_currency': 'EUR', 'from_currency': 'USD'},
   'id': '2a1679f3-365a-47fd-8533-64724a1461cd',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 139,
  'output_tokens': 13,
  'total_tokens': 152,
  'input_token_details': {'cache_read': 0}}}

In [17]:
# Message 3: ToolMessage
print("MESSAGE 3: ToolMessage")
print("=" * 70)
msg3 = result['messages'][2]
print(f"Tool Call ID: {msg3.tool_call_id}")
print(f"\nContent:")
print(msg3.content)

MESSAGE 3: ToolMessage
Tool Call ID: 2a1679f3-365a-47fd-8533-64724a1461cd

Content:
Conversion Result:
  1,000.00 USD = 920.00 EUR
  Exchange Rate: 1 USD = 0.9200 EUR


In [18]:
type(msg3)

langchain_core.messages.tool.ToolMessage

In [19]:
dict(msg3)

{'content': 'Conversion Result:\n  1,000.00 USD = 920.00 EUR\n  Exchange Rate: 1 USD = 0.9200 EUR',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'tool',
 'name': 'currency_converter',
 'id': '1631c80d-d16a-42fa-ada5-6d121c7ccb0c',
 'tool_call_id': '2a1679f3-365a-47fd-8533-64724a1461cd',
 'artifact': None,
 'status': 'success'}

In [20]:
# Message 4: Final AIMessage
print("MESSAGE 4: AIMessage (Final Response)")
print("=" * 70)
msg4 = result['messages'][3]
print(f"Has tool_calls: {bool(msg4.tool_calls) if hasattr(msg4, 'tool_calls') else False}")
print(f"\nContent:")
print(msg4.content)

MESSAGE 4: AIMessage (Final Response)
Has tool_calls: False

Content:
1000 USD is 920 EUR.


In [21]:
type(msg4)

langchain_core.messages.ai.AIMessage

In [22]:
dict(msg4)

{'content': '1000 USD is 920 EUR.',
 'additional_kwargs': {},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c32bc-4370-7103-974b-2abde54b9b3a-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 196,
  'output_tokens': 13,
  'total_tokens': 209,
  'input_token_details': {'cache_read': 0}}}

### Conversation Flow Visualization

Visual summary of the complete execution cycle.

In [23]:
# Visualize conversation flow
print("CONVERSATION FLOW VISUALIZATION")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER:")
        print(f"    \"{msg.content}\"")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 LLM → Calling tool:")
            for tc in msg.tool_calls:
                print(f"    Tool: {tc['name']}")
                print(f"    Args: {tc['args']}")
        else:
            print(f"\n[{i}] 🤖 LLM → Final response:")
            print(f"    \"{msg.content}\"")
            
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL RESULT:")
        for line in msg.content.split('\n'):
            print(f"    {line}")

print("\n" + "=" * 70)

CONVERSATION FLOW VISUALIZATION

[1] 👤 USER:
    "What is 1000 USD in EUR?"

[2] 🤖 LLM → Calling tool:
    Tool: currency_converter
    Args: {'amount': 1000, 'to_currency': 'EUR', 'from_currency': 'USD'}

[3] 🔧 TOOL RESULT:
    Conversion Result:
      1,000.00 USD = 920.00 EUR
      Exchange Rate: 1 USD = 0.9200 EUR

[4] 🤖 LLM → Final response:
    "1000 USD is 920 EUR."



## 6. Test Case 2: EMI Calculation

Test with a different tool to verify the pattern holds across different tool types.

In [24]:
# Test Case 2: EMI Calculator
state2 = {
    "messages": [
        HumanMessage(content="Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
    ]
}

print("Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
print("=" * 70)

result2 = app.invoke(state2)
print(f"\nMessage count: {len(result2['messages'])}")

Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months

Message count: 4


In [25]:
result2["messages"][-1].content

'OK. I have calculated the EMI for you. The monthly EMI is 1,555.31 USD. The total payment will be 55,991.19 USD, and the total interest paid will be 5,991.19 USD.'

### Tool Selection Verification

Verify that the LLM correctly selected the EMI calculator tool and extracted the appropriate parameters.

In [26]:
# Verify tool selection
print("Tool Selection Verification:")
print("=" * 70)
tool_call_msg = result2['messages'][1]
if tool_call_msg.tool_calls:
    tc = tool_call_msg.tool_calls[0]
    print(f"✅ LLM selected: {tc['name']}")
    print(f"\nExtracted parameters:")
    for key, value in tc['args'].items():
        print(f"   {key}: {value}")

Tool Selection Verification:
✅ LLM selected: emi_calculator

Extracted parameters:
   currency: USD
   annual_interest_rate: 7.5
   tenure_months: 36
   principal: 50000


In [27]:
# Display conversation flow for EMI test
print("\nCONVERSATION FLOW:")
print("=" * 70)

for i, msg in enumerate(result2["messages"], 1):
    if isinstance(msg, HumanMessage):
        preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
        print(f"\n[{i}] 👤 USER: \"{preview}\"")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 LLM: Calling {msg.tool_calls[0]['name']}")
        else:
            preview = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
            print(f"\n[{i}] 🤖 LLM: \"{preview}\"")
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL: Executed emi_calculator")


CONVERSATION FLOW:

[1] 👤 USER: "Calculate EMI for a 50000 USD loan at 7.5% for 36 ..."

[2] 🤖 LLM: Calling emi_calculator

[3] 🔧 TOOL: Executed emi_calculator

[4] 🤖 LLM: "OK. I have calculated the EMI for you. The monthly EMI is 1,555.31 USD. The tota..."


In [28]:
# Show final response
print("\nFINAL RESPONSE:")
print("=" * 70)
print(result2["messages"][-1].content)
print("=" * 70)


FINAL RESPONSE:
OK. I have calculated the EMI for you. The monthly EMI is 1,555.31 USD. The total payment will be 55,991.19 USD, and the total interest paid will be 5,991.19 USD.


## Conclusion

In this notebook, you learned:

✅ **Complete message flow** - A single tool execution creates 4 messages: HumanMessage (user query) → AIMessage with tool_calls (LLM decision) → ToolMessage (tool result) → AIMessage with content (final response)

✅ **Different message types** - Each message type serves a specific purpose: HumanMessage carries user input, AIMessage with tool_calls requests tool execution, ToolMessage returns results, AIMessage with content delivers the final answer

✅ **Execution cycle** - The graph flows through nodes: START → LLM (decides to call tool) → Tools (executes tool) → LLM (synthesizes response) → END

### Next Steps

Next, we'll explore **parallel tool execution** where the LLM suggests multiple tools and tools execute simultaneously in a single turn.